In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/news-summary/news_summary_more.csv
/kaggle/input/news-summary/news_summary.csv


### Import Essential Libraries

In [3]:
import numpy as np
import pandas as pd
import re

In [5]:
df=pd.read_csv('/kaggle/input/news-summary/news_summary.csv',encoding='iso-8859-1')

# -------- this data has a common encoding issue, it doesn't use utf-8 which is the default value ------

**BASIC EAD**

In [11]:
df

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...
...,...,...,...,...,...,...
4509,Mansha Mahajan,"24 Feb 2017,Friday",Rasna seeking ?250 cr revenue from snack categ...,http://indiatoday.intoday.in/story/rasna-eyes-...,Fruit juice concentrate maker Rasna is eyeing ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m..."
4510,Dishant Sharma,"03 Aug 2017,Thursday",Sachin attends Rajya Sabha after questions on ...,http://indiatoday.intoday.in/story/sachin-tend...,Former Indian cricketer Sachin Tendulkar atten...,Former cricketer Sachin Tendulkar was spotted ...
4511,Tanya Dhingra,"03 Aug 2017,Thursday",Shouldn't rob their childhood: Aamir on kids r...,http://www.hindustantimes.com/bollywood/secret...,"Aamir Khan, while talking about reality shows ...","Aamir Khan, whose last film Dangal told the st..."
4512,Pragya Swastik,"07 Dec 2016,Wednesday","Asha Bhosle gets ?53,000 power bill for unused...",http://indiatoday.intoday.in/story/singer-asha...,The Maharashtra government has initiated an in...,Maharahstra Power Minister Chandrashekhar Bawa...


In [7]:
df.shape

(4514, 6)

In [12]:
df.isnull().sum()

author         0
date           0
headlines      0
read_more      0
text           0
ctext        118
dtype: int64

In [14]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 6.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 9.6 MB/s eta 0:00:00


**Text preprocessing and cleaning**

In [16]:
import contractions # removes english's notation that makes word embedding more computationally expensive

def clean_text(text):
    text = str(text)  # ensuring input is string 
    text = text.lower()  # converting everyting lowercase
    text = contractions.fix(text)  # expand contractions
    text = re.sub(r'\([^)]*\)', '', text)  # remove brackets including text inside it
    text = re.sub('"','', text)          # remove all double quations
    text = re.sub(r"'s\b","",text)     #Removes possessive “’s” at the end of words.
    text = re.sub("[^a-zA-Z]", " ", text)   #Keeps only alphabets — removes numbers, punctuation, emojis, etc.
    text = re.sub('[m]{2,}', 'mm', text)   #Replaces multiple consecutive “m” letters with just “mm”.
    text = re.sub('\s+', ' ', text)   # removing multiple spaces
    return text
data=pd.DataFrame()
data['ctext'] = df['ctext'].apply(clean_text)
data['text'] = df['text'].apply(clean_text)

In [18]:
data['text'][0]

'the administration of union territory daman and diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of rakshabandhan on august the administration was forced to withdraw the decision within hours of issuing the circular after it received flak from employees and was slammed on social media '

In [19]:
df['text'][0] # here in the ouput we can see tha changes before and after text preprocssing

'The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7. The administration was forced to withdraw the decision within 24 hours of issuing the circular after it received flak from employees and was slammed on social media.'

In [20]:
max_text_words = max([len(list(text.split(" "))) for text in data['text']])
max_ctext_words = max([len(list(ctexts.split(" "))) for ctexts in data['ctext']])


print(max_text_words)
print(max_ctext_words)

69
11686


In [21]:
# Install Unsloth
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-x9_4on1r/unsloth_77b8a987b6d742bdb3a4f330d6fac319
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-x9_4on1r/unsloth_77b8a987b6d742bdb3a4f330d6fac319
  Resolved https://github.com/unslothai/unsloth.git to commit 98d82a769b6a4dbefb5638eb0c64e17c73363be1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 26.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.6 M

In [ ]:
# Import Unsloth libraries
from unsloth import FastLanguageModel
import torch
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [23]:
# Disable wandb completely
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

In [26]:
max_seq_length = 2048  # We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  


**Loading Pretraind Model with Unsloth**

In [27]:
# Load model - you can choose different models here
model, tokenizer = FastLanguageModel.from_pretrained(    #Loading pretrained model and its tokenizer.
    model_name = "unsloth/llama-3.2-1b-instruct-bnb-4bit", # You can choose any model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    
)

#converts text tokens (numerical IDs the model understands)

==((====))==  Unsloth 2025.10.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

**Integrating QLoRA adaptors for PEFT (Parameter-Efficient Fine-Tuning)**

In [28]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # LORa metrics Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],  #Only these layers are trainable; the rest of
                                                                #the model remains frozen.
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


Unsloth 2025.10.6 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


### Preparing Dataset for training **

In [32]:

def format_prompts(examples):
    texts = examples["text"]
    summaries = examples["ctext"]
    formatted_texts = []
    
    for text, summary in zip(texts, summaries):
        # Create instruction-following format
        prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize the following news article in a concise manner.

### Input:
{text}

### Response:
{summary}"""
        formatted_texts.append(prompt)
    
    return {"text": formatted_texts}

### Convert to HuggingFace dataset

In [36]:
dataset = Dataset.from_pandas(data)
dataset = dataset.map(format_prompts, batched=True)


Map:   0%|          | 0/4514 [00:00<?, ? examples/s]

In [38]:
dataset.shape

(4514, 2)

In [42]:
dataset

Dataset({
    features: ['ctext', 'text'],
    num_rows: 4514
})

### Spliting the dataset

In [40]:

train_dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_data = train_dataset["train"]
eval_data = train_dataset["test"]

print(f"Training samples: {len(train_data)}")
print(f"Evaluation samples: {len(eval_data)}")


Training samples: 4062
Evaluation samples: 452


In [47]:
from tqdm.auto import tqdm
# Training arguments
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    eval_dataset = eval_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        per_device_eval_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for longer training!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        save_steps = 100,
        eval_strategy = "steps",
        eval_steps = 100,
        save_total_limit = 2,
        load_best_model_at_end = True,
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        report_to = None,  # Disable wandb and other reporting
        disable_tqdm = False,  # Enable tqdm progress bars
        dataloader_num_workers = 0,  # Set to 0 for better tqdm display
    ),
)

# Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map (num_proc=2):   0%|          | 0/4062 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/452 [00:00<?, ? examples/s]

GPU = Tesla T4. Max memory = 14.741 GB.
14.281 GB of memory reserved.


In [48]:
# Check GPU usage
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Current device: {torch.cuda.current_device()}")
print(f"Device name: {torch.cuda.get_device_name()}")
print(f"Memory allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

CUDA available: True
Current device: 0
Device name: Tesla T4
Memory allocated: 1.17 GB


In [49]:
# Remove any wandb callbacks that might still be there
trainer.callback_handler.callbacks = [
    cb for cb in trainer.callback_handler.callbacks 
    if 'wandb' not in str(type(cb)).lower()
]

### Start training

In [50]:
from unsloth import unsloth_train
# unsloth_train fixes gradient_accumulation_steps
# trainer_stats = trainer.train()
trainer_stats = unsloth_train(trainer)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,062 | Num Epochs = 2 | Total steps = 508
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Step,Training Loss,Validation Loss
100,2.876400,2.839515
200,2.947500,2.798393
300,2.891000,2.780049
400,2.599100,2.772983
500,2.599300,2.770150


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


### Show final memory and time stats

In [51]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")


5676.49 seconds used for training.
94.61 minutes used for training.
Peak reserved memory = 14.303 GB.
Peak reserved memory for training = 0.022 GB.
Peak reserved memory % of max memory = 97.029 %.
Peak reserved memory for training % of max memory = 0.149 %.


### ============== INFERENCE AND TESTING ==============

In [53]:
# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Test the model
def generate_summary(text, max_length=150):
    prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize the following news article in a concise manner.

### Input:
{text}

### Response:
"""
    
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs, 
        max_new_tokens=max_length, 
        use_cache=True,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the response part
    response = response.split("### Response:")[1].strip()
    return response


In [54]:
# Test with a sample from the dataset
test_idx = 0
original_text = data.iloc[test_idx]['text']
original_summary = data.iloc[test_idx]['ctext']

print("Original Text:")
print(original_text[:500] + "...")
print("\nOriginal Summary:")
print(original_summary)
print("\nGenerated Summary:")
generated_summary = generate_summary(original_text)
print(generated_summary)



Original Text:
the administration of union territory daman and diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of rakshabandhan on august the administration was forced to withdraw the decision within hours of issuing the circular after it received flak from employees and was slammed on social media ...

Original Summary:
the daman and diu administration on wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media the union territory s administration was forced to retreat within hours of issuing the circular that made it compulsory for its staff to celebrate rakshabandhan at workplace it has been decided to celebrate the festival of rakshabandhan on august in this connection all offices departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall

### ============== SAVE MODEL ==============

In [56]:

# Save the model
model.save_pretrained("news_summarizer_Llama3.2-1B") # Local saving
tokenizer.save_pretrained("news_summarizer_Llama_tokenizer")

# Optional: Save to HuggingFace Hub (uncomment and add your token)
# model.push_to_hub("your_username/news_summarizer", token="your_hf_token")
# tokenizer.push_to_hub("your_username/news_summarizer", token="your_hf_token")

print("Model saved successfully!")

Model saved successfully!


In [ ]:
# Optional: Save to HuggingFace Hub (uncomment and add your token)
 model.push_to_hub("Rasel2091/news_summarizer_Llama-3.2_1B", token="HF write toke heren ")
 tokenizer.push_to_hub("your_username/news_summarizer_Llama-3.2_1B_tokenizer", token="HF write toke heren ")

IndentationError: unexpected indent (703456169.py, line 2)

In [ ]:
# Optional: Save to HuggingFace Hub (uncomment and add your token)
model.push_to_hub(
    "model/news_summarizer_Llama-3.2_1B", 
    token="HF write toke heren "
)

tokenizer.push_to_hub(
    "model/news_summarizer_Llama-3.2_1B_tokenizer", 
    token="HF write toke heren "
)


README.md:   0%|          | 0.00/598 [00:00<?, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Saved model to https://huggingface.co/Rasel2091/news_summarizer_Llama-3.2_1B


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

In [57]:
import shutil

# Path to your folder in Kaggle kernel
source_folder = "/kaggle/working/news_summarizer_Llama3.2-1B"

# Create a zip file
shutil.make_archive("/kaggle/working/news_summarizer_Llama3.2-1B", 'zip', source_folder)

print("Folder zipped successfully!")


Folder zipped successfully!


In [58]:
import shutil

# Path to your folder in Kaggle kernel
source_folder = "/kaggle/working/news_summarizer_Llama_tokenizer"

# Create a zip file
shutil.make_archive("/kaggle/working/news_summarizer_Llama_tokenizer", 'zip', source_folder)

print("Folder zipped successfully!")

Folder zipped successfully!
